### Load in JSON and check attributes

In [57]:
from eas_3d_pattern import SAMPLE_JSON, AntennaPattern

antenna_pattern = AntennaPattern(SAMPLE_JSON[0])  # load in a sample JSON file

Let's check the antenna pattern infos that we loaded it! We will print the antenna pattern which showcases the Info that are within the JSON file.

In [58]:
print(antenna_pattern)

===== Info =====
  File: 'VENDOR_ANTMODEL1_2100_A-Y1_RFp01_T00p0_P45_NonUniformSampling_3drp.json'
  Supplier: VENDOR
  Antenna Model: ANTMODEL1
  Antenna Type: Passive
  Revision Version: 1.0
  Released Date: 2023-05-05
  Coordinate System: SPCS_CW
  Beam ID: N/A
  Pattern Type: Generic
  Nominal Polarization: P45
  Optional Comments: example of passive antenna; theta and phi of radiation pattern are provided in Data_Set (non-uniform sampling)
==== Parameters ====
  Gain [dbi]: 15.11
  EIRP [dBm]: N/A
  Phi HPBW [deg]: 91.64
  Theta HPBW [deg]: 10.8
  Front to Back [db]: 21.72
==== Frequency & Tilt ====
  Frequency [Hz]: 2100000000.0
  Frequency Range [Hz]: N/A
  Theta Electrical Tilt [deg]: 0.0
  Phi Electrical Pan [deg]: N/A
==== Dataset Info ====
  Theta Sampling Range: N/A
  Phi Sampling Range: N/A
  Pattern Data Info: 
RangeIndex: 65160 entries, 0 to 65159
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  

The schema allows different units and structures of JSON files, so internally we construct a xarray dataset (similar to pandas dataframe) which adheres a standardized structure (e.g. always using the same coordintate system for plotting and calculations).

This structure contains mutliple data variables which are derived from Magnitude and Phase attached to Theta/Phi coordinates. It also saves some attributes like gain or hpbw that can be queried from it or directly from the *antenna_pattern* instance.

In [49]:
antenna_pattern.Pattern_3D

<xarray.Dataset> Size: 9MB
Dimensions:           (Theta: 181, Phi: 360)
Coordinates:
  * Theta             (Theta) int64 1kB 0 1 2 3 4 5 ... 175 176 177 178 179 180
  * Phi               (Phi) int64 3kB -180 -179 -178 -177 ... 176 177 178 179
Data variables: (12/15)
    MagAttenuationCo  (Theta, Phi) float64 521kB 87.36 87.37 ... 87.25 87.24
    MagAttenuationCr  (Theta, Phi) float64 521kB 129.0 110.7 ... 88.65 94.94
    MagAttenuationTP  (Theta, Phi) float64 521kB nan nan nan nan ... nan nan nan
    PhaseCo           (Theta, Phi) float64 521kB nan nan nan nan ... nan nan nan
    PhaseCr           (Theta, Phi) float64 521kB nan nan nan nan ... nan nan nan
    P_co_dB           (Theta, Phi) float64 521kB -87.36 -87.37 ... -87.25 -87.24
    ...                ...
    Phase_co_rad      (Theta, Phi) float64 521kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    Phase_cr_rad      (Theta, Phi) float64 521kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    E_co_complex      (Theta, Phi) complex128 1MB (4.285218919915373e-05+0j) ...
    E_cr_complex      (Theta, Phi) complex128 1MB (3.5406253329726707e-07+0j)...
    P_tp_lin          (Theta, Phi) float64 521kB 1.836e-09 ... 2.21e-09
    P_tp_dB           (Theta, Phi) float64 521kB -87.36 -87.35 ... -84.89 -86.56
Attributes:
    gain_dbi:           15.11
    phi_hpbw:           91.64
    theta_hpbw:         10.8
    front_to_back:      21.72
    coordinate_system:  SPCS_Ericsson

In [50]:
print(f"Antenna Pattern gain in dBi: {antenna_pattern.gain_dbi}")  # gain in dBi
antenna_pattern.Pattern_3D["P_tp_dB"]  # power pattern in dB

Antenna Pattern gain in dBi: 15.11


<xarray.DataArray 'P_tp_dB' (Theta: 181, Phi: 360)> Size: 521kB
array([[-87.36024325, -87.34539066, -87.30611311, ..., -87.25749257,
        -87.31583506, -87.35032629],
       [-76.39943066, -76.28362337, -75.98194184, ..., -75.60574497,
        -76.03655504, -76.31300678],
       [-70.40536028, -70.28539376, -69.97336035, ..., -69.58511924,
        -70.02969876, -70.31577518],
       ...,
       [-70.37411873, -70.18342319, -69.70047742, ..., -69.12197589,
        -69.78480845, -70.23066514],
       [-76.36074117, -76.07168565, -75.36597791, ..., -74.56156958,
        -75.48543283, -76.14202908],
       [-87.21715462, -86.35902938, -84.62190012, ..., -83.01270484,
        -84.88536718, -86.55536885]], shape=(181, 360))
Coordinates:
  * Theta    (Theta) int64 1kB 0 1 2 3 4 5 6 7 ... 174 175 176 177 178 179 180
  * Phi      (Phi) int64 3kB -180 -179 -178 -177 -176 ... 175 176 177 178 179

### Calculate beam efficiency

Sectors and methods to caluclate beam efficiency can differ. You can get familiarize yourself with the Ericsson EAS methodology here: [Example of Beam Efficiency Report / EAS Definitions](https://erilink.internal.ericsson.com/eridoc/erl/objectId/09004cffd60af4fb?docno=2%2F0363-KRE2014818%2F21&option=download&format=pdf)

A simplistic summation method is proposed here. Furthermore, EAS sectors are used as default.

The output is a dictionary with the name of the sectors and the corresponding ratios to the total power.

In [51]:
antenna_pattern.calculate_beam_efficiency()

{'Cell': 0.7299576611340302,
 'Int1': 0.06328372416549503,
 'Int2': 0.06322207683325005,
 'Int3': 0.1094309953021251,
 'EMF': 5.8186354524159285e-05,
 'Wasted': 0.0340473562105757}

In [52]:
print(
    f"The beam efficiency of the main sector is: {antenna_pattern.calculate_beam_efficiency()['Cell'] * 100:.2f}%"
)

The beam efficiency of the main sector is: 73.00%


Now we want to calucate the beam efficiency in a different sector.

For this we import the *SectorDefinition* class and add a sector. The sector can be any rectangular shape.

Note: We use always the reference coordinate system with Theta range 0...180° and Phi range -180°...179°

In [53]:
from eas_3d_pattern import SectorDefinition

In [54]:
my_sectors = SectorDefinition(load_default=False)
my_sectors.add_sector(
    name="left_beam_until_horizon",
    theta_min=(
        90.0,
        "<",
    ),  # "<" or "<=" allowed. 90° and below, meaning it starts with this data at 91°
    theta_max=(165.0, "<="),
    phi_min=(-60.0, "<="),
    phi_max=(0.0, "<="),
)
my_sectors.add_sector(
    name="right_beam_until_horizon",
    theta_min=(90.0, "<"),
    theta_max=(165.0, "<="),
    phi_min=(0.0, "<="),
    phi_max=(60.0, "<="),
)

In [55]:
print(my_sectors)

SectorDefinition (2 defined Sectors)
'left_beam_until_horizon': 	[90.0<Theta<=165.0], [-60.0<=Phi<=0.0]
'right_beam_until_horizon': 	[90.0<Theta<=165.0], [0.0<=Phi<=60.0]


In [56]:
antenna_pattern.calculate_beam_efficiency(sector_definitions=my_sectors)

{'left_beam_until_horizon': 0.19818456046824967,
 'right_beam_until_horizon': 0.19797713481383544}